In [7]:
import pygame, sys
from pygame.locals import *
import os
import numpy
import matplotlib

import qiskit
from qiskit import *

matplotlib.use("Agg")

import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg

pygame.init()
level1 = pygame.image.load("../level1/level1.jpg") #credit: <a href="/free-image/blue-pixels-wallpaper-9124.html">Blue Pixels Wallpaper</a>
level1_bg = pygame.transform.scale(level1, (800, 600))

mx, my = pygame.mouse.get_pos()

In [9]:
def level1_intro(x, y, xChange, yChange):
    running = True
    duck_button = pygame.Rect(0,0,64,64)
    jump = False
    fall = False
    inst = True
    i = 0
    xChange = 0
    yChange = 0
    y = 450
    x = 100
    bg_movement = 0
    width = 800

    while running:
        
        screen.fill((0,0,0))
        screen.blit(level1, (0,0))
        # Setup
        mx, my = pygame.mouse.get_pos()
        
        instructions = ["It's time to learn about quantum circuits!", 
                       "Circuits are a great way to learn about elementary quantum computations",
                       "These gates operate on different qubits",
                       "If you ever want to start over and come back here, press ESC",
                       "To go to the next stage, press the 'N' button"]
        #Instructions
        if i == 0:
            if inst:
                header_rect = draw_text("Use the SPACE Button to continue reading...", fontSubHeader, (0, 0, 0), screen, 50, 200)
        text_rect = draw_text(instructions[i], text, (0, 0, 0), screen, 50, 400)
    
        #Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                # movement
                if event.key == pygame.K_LEFT:
                    xChange = -5
                if event.key == pygame.K_RIGHT:
                    xChange = 5
                if event.key == pygame.K_UP:
                    yChange = -5
                if event.key == pygame.K_DOWN:
                    yChange = 5
                if event.key == pygame.K_SPACE and i < len(instructions)-1:
                    i += 1
                elif event.key == pygame.K_SPACE and i == len(instructions)-2:
                    print(here)
                    inst = False
                    i = 0
                elif event.key == pygame.K_n:
                    level1_example_circ(x, y, xChange, yChange)
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    xChange = 0
                if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    yChange = 0
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
        
        
        #fix movement
        x += xChange
        if x <= 0:
            x = 0
        elif x >= 800-duck_button.width:
            x = 800-duck_button.width
        
        y += yChange
        if y <= 0:
            y = 0
        elif y >= 600-duck_button.height:
            y = 600-duck_button.height
        
        #Display
        duck_button = Icon(icon, x, y)
        pygame.display.update()
        mainClock.tick(60)